# Particle Image Velocimetry (PIV) Walkthrough  #

In [ ]:
# Import packages image processing / feature matching / plotting
from os.path import normpath, join
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage import io
import cv2 as cv2
from skimage.feature import match_template
import seaborn as sns

## Read in your initial image and set the parameters for the PIV ##

In [ ]:
im_cb = io.imread('/.../imageStack.tif') # load cell image (this will be a .tif stack)
index = 55 # choose a frame from the time-lapse movie

img1 = im_cb[index]
img2 = im_cb[index+1]

In [ ]:
sourceSize = 10 # how big are the features you're tracking (in pixels)?
searchSize = 20 # make a bigger box (usually around double the source size)
gridDistance = 5 # define the grid (can overlap)
correlationThreshold = 0.5 # for noisy images choose a lower value
interval = 5.0 # define the time interval between images (in seconds)
fps = 1/interval 
mue2pix_ratio = 0.1 # define the pixel/micron calibration

In [ ]:
i=313 # choose example coordinates within the actin network
j=193 

sourceImage = img1[i-sourceSize:i+sourceSize, j-sourceSize:j+sourceSize] # define the source sub image
searchImage = img2[i-searchSize:i+searchSize, j-searchSize:j+searchSize] # define the search sub image      

### This section displays the source image that will be matched against the search image ###

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, figsize=(15, 15))

ax1.imshow(img1, "gray", origin='upper')
ax1.set_axis_off()
ax1.set_title('Frame t')

# highlight matched region
h1, w1 = sourceImage.shape
rect1 = plt.Rectangle((j-h1/2.0, i-w1/2.0), w1, h1, edgecolor = sns.xkcd_rgb["neon green"], facecolor='none')
ax1.add_patch(rect1)

ax2.imshow(sourceImage, "gray", origin='upper')
ax2.set_axis_off()
ax2.set_title('Source Image')

ax3.imshow(img2, "gray", origin='upper')
ax3.set_axis_off()
ax3.set_title('Frame t+1')

# highlight matched region
h, w = searchImage.shape
rect = plt.Rectangle((j-h/2.0, i-w/2.0), w, h, edgecolor = sns.xkcd_rgb["neon green"], facecolor='none')
edgecolor = sns.xkcd_rgb["neon green"]
ax3.add_patch(rect)

ax4.imshow(searchImage, "gray", origin='upper')
ax4.set_axis_off()
ax4.set_title('Search Image')
# plt.savefig('/.../PIV Sequence{}.pdf'.format(index), bbox_inches='tight', pad_inches=0)

plt.show()

### Now we do the cross correlation between search and source images, and checks for a good match ###

In [ ]:
c = match_template(searchImage, sourceImage) # run the image 2d cross correlation

ypeak, xpeak = np.unravel_index(np.argmax(c), c.shape) # find the location of the maximum correlation coefficent
corr_val = np.max(c) 

if corr_val > correlationThreshold :

    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(10, 10))

    ax1.imshow(-1*sourceImage, "gray", origin='upper')
    ax1.set_axis_off()
    ax1.set_title('Source Image')

    ax2.imshow(searchImage, "gray", origin='upper')
    ax2.set_axis_off()
    ax2.set_title('Search Image')
    y,x = searchImage.shape
    cx = x/2
    cy = y/2

    h, w = sourceImage.shape
    rect = plt.Rectangle((cx-h/2, cy-w/2), w, h, edgecolor=sns.xkcd_rgb["neon green"], facecolor='none')
    ax2.add_patch(rect)

    ax3.imshow(c,cmap='viridis')
    ax3.set_axis_off()
    ax3.set_title('Correlation Field')

#   plt.savefig('/.../PIV Correlation{}.pdf'.format(index), bbox_inches='tight', pad_inches=0)
    plt.show()

## This bit will the calculate our example displacement vectors for the source image ##

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 10))

ax1.imshow(searchImage)
ax1.set_axis_off()
ax1.set_title('Search Image')

cy,cx = [searchImage.shape[0]/2,searchImage.shape[1]/2]
h, w = sourceImage.shape

rect = plt.Rectangle((cx-h/2, cy-w/2), w, h, edgecolor=sns.xkcd_rgb["white"], facecolor='none')
ax1.add_patch(rect)

ax1.quiver(cx, cy, dx, dy, color = sns.xkcd_rgb["neon green"], angles='xy') 

ax2.imshow(img1, origin='upper')
ax2.quiver(x_p, y_p, dx, dy, color = sns.xkcd_rgb["neon green"], angles='xy')
ax2.axis('off')

plt.show()

## The next section will calculate velocity vectors for all source images within the cell ##

In [ ]:
img1 = im_cb[index]
img2 = im_cb[index+1]

M, N = img1.shape[0], img1.shape[1] # image size

kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

# Load current frame
cellOutline1 = cv2.Canny(img1, 0, 1)

# Dilate image edges
im_dilation1 = cv2.dilate(cellOutline1, kernel1, iterations = 1)

# Fill holes in the dilated image
outline1 = np.zeros([im_dilation1.shape[0]+2, im_dilation1.shape[1]+2], dtype = np.uint8)
cv2.floodFill(im_dilation1, outline1, (0, 0), 255)

# Erode image to find cell outline
cv2.erode(cellOutline1, outline1, kernel1, iterations = 1)
cellOutline1 = 1 - outline1[1:-1,1:-1]

kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (searchSize, searchSize))
img_erode1 = cv2.erode(cellOutline1,kernel2,iterations = 1)

# Load current frame
cellOutline2 = cv2.Canny(img2, 0, 1)

# Dilate image edges
im_dilation2 = cv2.dilate(cellOutline2, kernel1, iterations = 1)

# Fill holes in the dilated image
outline2 = np.zeros([im_dilation2.shape[0]+2, im_dilation2.shape[1]+2], dtype = np.uint8)
cv2.floodFill(im_dilation2, outline2, (0, 0), 255)

# Erode image to find cell outline
cv2.erode(cellOutline2, outline2, kernel1, iterations = 1)
cellOutline2 = 1 - outline2[1:-1,1:-1]

img_erode2 = cv2.erode(cellOutline2,kernel2,iterations = 1)

interSection = img_erode1 * img_erode2

x_p = []
y_p = []
dx = []
dy = []
c_val = []
 
for i in range(0, M, gridDistance):
    for j in range(0, N, gridDistance):
        if interSection[i,j] == 1:
            sourceImage = img1[i-sourceSize:i+sourceSize, j-sourceSize:j+sourceSize]
            searchImage = img2[i-searchSize:i+searchSize, j-searchSize:j+searchSize]        
            
            c = match_template(searchImage, sourceImage)
                        
            ypeak, xpeak = np.unravel_index(np.argmax(c), c.shape)
            
            corr_val = np.max(c) # if corr > 0.5
            
            if corr_val > correlationThreshold:
               
                x_p.append((j))
                y_p.append((i))
                dx.append((xpeak - (c.shape[0] + 1) / 2))
                dy.append((ypeak - (c.shape[1] + 1) / 2))
                c_val.append((corr_val))
        
vx = [i * mue2pix_ratio * fps * 60 for i in dx] 
vy = [i * (mue2pix_ratio * fps * 60) for i in dy]
s0 = np.sqrt(np.array(vx)**2 + np.array(vy)**2) 

plt.imshow(img1, "gray", origin='upper')
plt.quiver(x_p, y_p, vx, vy, s0,
           cmap=plt.cm.jet, angles='xy') 
    
plt.axis('off')
plt.clim(0,12)
    
# plt.savefig('/.../flows00{}.png'.format(index), bbox_inches='tight', pad_inches=0, dpi=600)
plt.show()

plt.close()